<a href="https://colab.research.google.com/github/NunezKant/COVID-19_GDL/blob/master/Desarrollo_de_COVID_19_en_Guadalajara_Miguel_N%C3%BA%C3%B1ez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Esta celda importa librerías que necesitaremos { display-mode: "form" }
%matplotlib inline
!pip install arviz
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import arviz as az
import pymc3 as pm
import altair as alt
import plotly.express as px
import requests
import io

sns.set_context('talk')
plt.style.use('seaborn-whitegrid')

MIT License

Copyright (c) 2020 Miguel Núñez 

<h1><center> Desarrollo de COVID - 19 en Jalisco </center></h1>

#Contexto

Hola! Me alegra que estés aquí, juntos vamos a aprender un poco sobre modelaje epidemiólogico y la situación del COVID-19 en Jalisco.

Mi nombre es Miguel Angel Núñez Ochoa, soy estudiante de doctorado en Neurociencias, y de un micromaster en estadística y ciencia de datos.

Formo parte de [DataLab Community](https://www.facebook.com/datalabmx) la comunidad de ciencia de datos más grande de Guadalajara, como mentor en ciencia de datos.

>>>>>![DataLab](https://drive.google.com/uc?id=0B3a3JGJVqFAAdFNETHdhVktFWFk)

Además de ser organizador y profesor en la escuela de verano de la Reunión Internacional de inteligencia Artificial y sus aplicacciones ([RIIAA](https://riiaa.org/)).

>>>>>![RIIAA](https://drive.google.com/uc?id=1R1dJbJgCxTME5jbYBxV8p6tnbN1F1yHT)

No soy experto en el área de epidemiología, pero tendrán las referencias de dónde estoy sacando los parámetros, o los modelos.

La motivación para hacer esto es por la desinformación que ronda en redes sociales y el poco contenido en español, esta es una libreta interactiva, la plataforma se llama Google Colab, este tipo de libreta nos permite escribir texto como el que estás viendo, intercalado con código, al lado de las celdas verán un simbolo de play, en caso de ser una celda de código, al darle click, ejecutan el codigo dentro de esa celda.

**Cualquier error u observación hagánmela saber**

Primero nos centraremos en entender el modelado epidemiológico de enfermedades, visitaremos algunos conceptos claves y veremos que nos pueden decir los modelos, trataré de hacer lo más digerible posible las partes que involucren muchas matemáticas.

---
#Introducción al modelo SEIR

El modelo con el cuál vamos a trabajar es conocido como **SEIR** y la implementación que hice está basada en el libro:
 * *Modeling Infectious Diseases in humans and animals (Keeling and Rohani, 2008)* 

Este modelo es uno de los modelos epidemiológicos más sencillos, se llama así por sus siglas,que hacen referencia a la capacidad del portador para transmitir la enfermedad:

* **S**uceptible: Persona aún no expuesta al patógeno, pero forma parte de una población que ya tiene expuestos.

* **E**xpuesto: Aquellas personas que están infectados pero aun no son *infecciosos*
 * Estas personas ya tienen el virus, pero el virus se encuentra en una etapa de desarrollo en el cuerpo, la carga viral es tán poca que la transmisión activa del virus es poco probable.
* **I**nfectados: Personas colonizadas por el patógeno
* **R**ecuperados: Aquellos que ya resolvieron la enfermedad.

>>>>![SEIR](https://drive.google.com/uc?id=16GNLZ5a8zZctmzWjMECjKrUZ4Zn4hwwQ)

---
#MATHBOX
<h2><center>¿Cómo modelamos esto?</center></h2>

Al ser un sistema dinámico, evoluciona con el tiempo, este tipo de sistemas se modelan con ecuaciones diferenciales, en el caso específico del modelo **SEIR** las ecuaciones son las siguientes:

\begin{gather*} 
\dot{S} =  - \beta SI \\ 
\dot{E} =  \beta SI - \alpha E \\
\dot{I} =  \alpha E - \gamma I \\ 
\dot{R} =  \gamma I \\
S+E+I+R = 1
\end{gather*}

1. La primera ecuación describe el cambio en personas suceptibles
 * $\beta\$ es el nombre que le daremos a la **tasa promedio de contagio**

<font color='red'>FACT:</font> el distanciamiento social y la cuarentena son inversamente proporcionales a $\beta\$, es decir:

<center> "Al aumentar el distanciamento social, disminuimos la tasa promedio de contagio, por que reducimos el número de individuos infectados que se están mezclando de manera libre en la población" </center>

2. La segunda ecuación describe a las personas expuestas:
 * $\alpha$ es el **inverso del periodo de incubación**
 * Según (Laurer et al. 2020; Hellewell et al. 2020; Guang et al. 2020) el periodo de incubación para COVID-19 es de alrededor de **5 dias**, por lo tanto:
  * $\alpha = 1 / 5 \rightarrow 0.2$

3. La tercera ecuación modela el cambio en personas infectadas, basada en la gente expuesta (aquellas infectadas pero que aun no son *infecciosas*) incubando el virus y la gente infectada en vías de recuperación.

 * $\gamma$ es la **tasa de recuperación**, y su recíproco el **periodo infeccioso promedio** o **tiempo latente**

4. La cuarta ecuación modela a los recuperados.

# Paréntesis: ¿Han escuchado el término $R_0$?

Este término se vuelve muy popular en tiempos de crisis epidemiológicas, también llamado **tasa básica de reproducción** se define formalmente como:

<center> "el número promedio de casos secundarios que tienen lugar a partir de un caso primario en una población suceptible" </center>

Y está dada por dos cantidades que ya conocemos $\beta$ y $\gamma$:

\begin{gather*} 
R_0 = \frac{\beta}{\gamma}
\end{gather*}

Es decir, es un cociente entre la capacidad del virus para contagiar y la tasa de recuperación que representa dicho virus.

Conocer esta ecuación nos ayudará a encontrar $\beta$:
 * De (Peng et al. 2020) podemos tomar el estimado de $\gamma$ para COVID-19: $0.5$
 * De (Hellewell et al. 2020) podemos tomar el estimado de $R_0$ del COVID-19:
 $3.5$

Sustituyendo obtenemos que $\beta$ es $\sim 1.8$ 

*Algo a tener en cuenta es que este modelo asume una **población fija** es decir, no toma en cuenta nacimientos ni muertes, en el mejor de los casos podemos interpretar como que los decesos se acumulan en las personas "recuperadas" por que ya no pueden transmitir la enfermedad.*

---

<h1><center>Caso especifíco de Jalisco México</center></h1>

(22/03/2020)

Vamos a usar este modelo para ejemplificar la situación del estado de Jalisco, la cual:

* Tiene una población ($N$) de $\sim$ 8,000,000 habitantes.
* Tiene un total de 32 infectados ($I$)
* Asumiremos que como **mínimo** esas 32 personas han expuesto a 4 más (familias de 4 personas)
* Haremos la evaluación de la evolución de la enfermedad por 90 días.

In [1]:
dias_evaluacion = 90 #asumimos 90 días
dt = 1
periodo_evaluacion = np.linspace(0, dias_evaluacion, dias_evaluacion + 1)

alpha = 0.2
beta = 1.75
gamma = 0.5
parametros = alpha, beta, gamma

#Condiciones iniciales de la ZMG
JAL_Population = 8000000 
I_o = 32 / JAL_Population  # Tenemos 32 casos
E_o = (32*4)/ JAL_Population # Asumimos 4 expuestos por caso
S_o = (1) - (E_o+I_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado

Condiciones_Iniciales = S_o,E_o,I_o,R_o

NameError: ignored

In [0]:
def SEIR(IC,pmtrs,t,theta=1):
  """
  Esta parte de código es la que implementa las ecuaciones diferenciales.

  Se resuelven por medio de integración númerica de Euler
  """
  
  So,Eo,Io,Ro = IC
  S,E,I,R = [So],[Eo],[Io],[Ro]
  alpha,beta,gamma = pmtrs
  dt = t[1]-t[0]
  for _ in t[1:]:
    St = S[-1] - (theta*beta*S[-1]*I[-1])*dt
    Et = E[-1] + (theta*beta*S[-1]*I[-1] - alpha*E[-1])*dt
    It = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
    Rt = R[-1] + (gamma*I[-1])*dt
    S.append(St)
    E.append(Et)
    I.append(It)
    R.append(Rt)
  return S,E,I,R

In [0]:
S,E,I,R = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion) # Obtenemos los suceptibles, expuestos, infectados y recuperados para los proximos 90 dias

In [0]:
#@title Esta Celda grafica los resultados { display-mode: "form" }

E_a = np.array(E)*JAL_Population
I_a = np.array(I)*JAL_Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0])
Dias = np.concatenate([np.arange(0,91,1),np.arange(0,91,1)])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase')

fig.update_layout(
    title="Predicción del modelo SEIR de la evolución de COVID-19 en Jalisco, Mexico",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

El dia 0 es hoy 22/03/2020, si se quiere correr el análisis después habrá que cambiar las condiciones inciales (actualizar en número de infectados reportados)

# ¿Cómo saber cuando actuar?

Para tratar de responder esto, ajustaré una recta a los primeros 21 días, después, buscaré el punto de inflexión de los residuos de esa linea (qué sería igual a buscar el día en que el comportamiento lineal se perdío).

In [0]:
#@title Ajuste de la linea

import statsmodels.api as sm

X = SEIR_df["Dias"][:21].values
y = SEIR_df["Casos"][SEIR_df["Clase"]=="Infectados"][:21].values

model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

In [0]:
#@title ¿En donde empieza a equivocarse más la recta?

plt.figure(figsize=(9,5))
plt.plot(results.resid, alpha = 0.5)
plt.scatter(np.argmin(results.resid),np.min(results.resid),c = "r")
plt.ylabel("Residuos del ajuste lineal")
plt.xlabel("Dias")
dia = np.argmin(results.resid)
plt.title(f"El punto de inflexión es al día: {dia} ")

Es probable que el día crítico suceda en 11 días, que será cúando el comportamiento lineal empieza a subestimar los casos.

Ese día a día de hoy sería alrededor del 2 al 3 de abril, así que por favor:

<h1><center>QUEDENSE EN SU CASA</center></h1>
<h1><center>NI PENSAR EN VACACIONES!</center></h1>

Si tomamos en cuenta el día que se estima el pico máximo de casos (Día 50), y el día de inflexión (día 11) el modelo calcula 39 días de contigencia, lo equivalente a $\sim 6$ semanas aproximadamente.

Por lo que la contención debería estar pensada como mínimo que vaya de entre <font color='red'>3-7 semanas</font> lo que concuerda con recomendaciones internacionales, dado que esta estimación no toma en cuenta el efecto del aislamiento social.

Por otro lado, para observar el efecto del aislamiento y distanciamiento social, para los que observaron la parte del código de las ecuaciones diferenciales:

```
    St = S[-1] - (theta*beta*S[-1]*I[-1])*dt
    Et = E[-1] + (theta*beta*S[-1]*I[-1] - alpha*E[-1])*dt
```

A las ecuaciones de **S**uceptibles y **E**xpuestos, les agregué un parámetro que afecta a $\beta$ en este caso lo llamé $\theta$, este parámetro simula la proporcion de gente que atiende la cuarentena (**DISTANCIAMIENTO**).

$\theta = 0 \rightarrow$ supone que todos hacemos caso absoluto a la cuarentena

$\theta = 1 \rightarrow$ supone que todos hacemos caso omiso a la cuarentena

La ecuación modificada quedaría de la siguiente forma:


\begin{gather*} 
\dot{S} =  - \theta \beta SI \\ 
\dot{E} = \theta \beta SI - \alpha E \\
\end{gather*}





In [0]:
#@title AISLAMIENTO SOCIAL VS CASO OMISO (Interactivo!)



import plotly.graph_objects as go
fig = go.Figure()

for step in np.round(np.arange(0, 1.1, 0.1),1):
  S,E,I,R = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion,1-step)
  fig.add_trace(
        go.Scatter(
            visible=False,
            showlegend = True,
            line=dict(color="#DE1738", width=2),
            name="Infectados",
            x=np.arange(0, 91, 1),
            y=np.array(I)*JAL_Population)
        )
  

fig.data[0].visible = True

steps = []
labels = np.round(np.arange(1.0, -0.1, -0.1),1).tolist()
labels[0] = "Caso Omiso"
labels[10] = "Aislamiento Total"
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
        label = str(labels[i])
    )
    step["args"][1][i] = True  
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Distanciamiento (Theta): "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="Predicción del modelo SEIR de la evolución de COVID-19 en Jalisco, Mexico",
    xaxis_title="Días",
    yaxis_title="Infectados Totales",
)

fig.show()

Como podemos ver el atender las recomendaciones de salubridad, lavarnos las manos, salir de casa exclusivamente por cuestiones prioritarias como una emergencia, salir por víveres o medicamentos, es una forma efectiva de aplanar la curva de infectados, descargar hospitales y poner al menor número de personas en riesgo.

Hay que atender las recomendaciones! <3 

In [0]:
#@title QUEDATE EN CASA { display-mode: "form" }

periodo_evaluacion = np.linspace(0, 90, dias_evaluacion + 1)
_,E_1,I_1,_ = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion)
_,E_5,I_5,_ = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion,0.7)
_,E_3,I_3,_ = SEIR(Condiciones_Iniciales,parametros,periodo_evaluacion,0.3)
plt.figure(figsize= (16,6))
plt.suptitle("Contribuye a aplanar la curva!")
plt.plot(E_1,label = "Expuestos, theta = 1", ls = "--",alpha=0.3,c="#FF0000")
plt.plot(I_1,label = "Infectados, theta = 1",c="#FF0000")
plt.plot(E_5,label = "Expuestos, theta = 0.5", ls = "--",alpha=0.3,c="#FFCC00")
plt.plot(I_5,label = "Infectados, theta = 0.5",c="#FFCC00")
plt.plot(E_3,label = "Expuestos, theta = 0.3", ls = "--",alpha=0.3,c="#99CC00")
plt.plot(I_3,label = "Infectados, theta = 0.3",c="#99CC00")
plt.ylabel("Proporcion de la población")
plt.xlabel("Dias")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

<h1><center>¿Qué pasa con los asintomáticos?</center><h1>

Para tratar de responder a esta pregunta, tomé el modelo propuesto en:

> Mathews, J. D., McCaw, C. T., McVernon, J., McBryde, E. S., & McCaw, J. M. (2007). A biological model for influenza transmission: pandemic planning implications of asymptomatic infection and immunity. PLoS One, 2(11).

Lo simplifiqué y lo estoy modelando con los mismos parámetros comentados anteriormente.

El modelo SEAIR (por la inclusión de los **A**sintomáticos) quedaría formalmente descrito de la siguiente forma:

\begin{gather*} 
\dot{S} =  - \beta SI \\ 
\dot{E} =  \beta SI - \alpha E \\
\dot{A} =  (1-\rho)\cdot\alpha E - \gamma A \\
\dot{I} =  \rho \cdot \alpha E - \gamma I \\ 
R = 1 - (S+E+I+A)
\end{gather*}

Me permitiré traducirles un segmento del paper que me parece fundamental:

> "$R_o$ es la condición umbral para una epidemia en una población completamente suceptible y en su forma más general, se puede describir como el número de exposiciones (que llevan a ambos casos (**A** e **I**)) por transmisor (**A** o **I**).

> Dada la falta de información detallada sobre quién infectó a quién, la incidencia de repotes de síntomas en la población no provee información sobre la capacidad infecciosa de transmisores sin sintomas.

> Por ello, nosotros asumimos una proporción de todos los infectados latentes como sintomáticos (**I**) (en esta implementación yo le llamé a esa proporción $\rho$) y el complemento ($1-\rho$) aquellos portadores sin síntomas (**A**).

> Sobre el supuesto que ambos portadores (**I** e **A**) tienen la misma $\gamma$  (inverso de **tiempo promedio de capacidad infecciosa**)

<h3><center>Es de notar que si algunos casos asintomáticos no son reportados de manera aleatoria, entonces, estos son implícitamente agregados a los casos asintomáticos. Por ello, el grupo asintomático podría describirse propiamente como asintomático o no reportado"</center></h3>

Hay hasta el momento $4$ casos sin síntomas en Jalisco, de un total de 45 (23/03/2020) lo que representa una proporción $\sim 10 \% \rightarrow \rho = 0.9$.

Sin embargo el número de muestras para calcular esa proporción puede estar sub-estimada, dado el bajo número de muestras realizadas, por lo que se hizo un estudio de la literatura para encontrar esta proporción, de multiples trabajos del autor (Mizumoto et al. 2020) se ha estimado que la proporción de pacientes asintomáticos ronda el $\sim 18 \% \rightarrow \rho = .82$. 
 
Todo lo demás lo seguiré modelando con los casos reportados a día de (22/03/2020) para que el notebook tenga coherencia.

Se trata del modelo base que asume una $\theta = 1$, es decir sin distanciamiento social.





In [0]:
dias_evaluacion = 90 #asumimos 90 días
periodo_evaluacion = np.linspace(0, dias_evaluacion, dias_evaluacion + 1)

alpha = 0.2
beta = 1.75
gamma = 0.5
rho = 0.82
parametros = alpha, beta, gamma, rho

JAL_Population = 8000000 
I_o = 32 / JAL_Population  # Tenemos 32 casos
E_o = (32*4)/JAL_Population # Asumimos 4 expuestos por caso
A_o =  4 / JAL_Population
S_o = (1) - (E_o+I_o+A_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado

Condiciones_Iniciales = S_o,E_o,A_o,I_o,R_o,

In [0]:
def SEAIR(IC,pmtrs,t,theta=1):
  """
  Esta parte de código es la que implementa las ecuaciones diferenciales.

  Se resuelven por medio de integración númerica de Euler
  """
  
  So,Eo,Ao,Io,Ro = IC
  S,E,A,I,R = [So],[Eo],[Eo],[Io],[Ro]
  alpha,beta,gamma,rho = pmtrs
  dt = t[1]-t[0]
  for _ in t[1:]:
    St = S[-1] - (theta*beta*S[-1]*I[-1])*dt
    Et = E[-1] + (theta*beta*S[-1]*I[-1] - alpha*E[-1])*dt
    At = A[-1] + (1-rho)*(alpha*E[-1] - gamma*A[-1])*dt
    It = I[-1] + rho*(alpha*E[-1] - gamma*I[-1])*dt
    Rt = 1 - (S[-1]+E[-1]+A[-1]+I[-1])
    S.append(St)
    E.append(Et)
    A.append(At)
    I.append(It)
    R.append(Rt)
  return S,E,A,I,R

In [0]:
S,E,A,I,R = SEAIR(Condiciones_Iniciales,parametros,periodo_evaluacion) # Obtenemos los suceptibles, expuestos, infectados y recuperados para los proximos 90 dias

In [0]:
#@title Resultados modelo SEAIR { display-mode: "form" }

E_a = np.array(E)*JAL_Population
I_a = np.array(I)*JAL_Population
A_a = np.array(A)*JAL_Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ ["Asintomaticos o No reportados"]*np.array(E).shape[0])
Dias = np.concatenate([np.arange(0,91,1),np.arange(0,91,1),np.arange(0,91,1)])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase')

fig.update_layout(
    title="Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Mexico",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

El modelo resalta la importancia de aumentar las pruebas per capita. 

> Conforme se pueda contener el número de casos confirmados, será vital aumentar el número de pruebas per capita, ya qué en un punto los asintomáticos o no reportados superan los casos confirmados, si no se aumenta la capacidad de screening de la población, se pudiera observar un fenómeno cíclico.

<h1><center>¿Cúantos hospitalizados y muertes pudiera haber en el peor escenario?</center></h1>

Para responder esto simplemente agregaremos al modelo **SEAIR** una nueva ecuación:

\begin{gather*} 
\dot{D} =  \kappa I
\end{gather*} 

Siendo $\kappa$ la tasa de moratalidad, después de una busqueda, parece que este parametro pudiera depender del tiempo, en relación a la respuesta del sistema de salud, y de caracteristicas de la población (co-morbilidades y randos de edad).

Sus rangos estar entre $.1 \%$ a $3 \%$ (Peng et al. 2020).

Sin embargo para este ejercicio, tomaré una tasa constante de $1.6 \%$.


In [0]:
dias_evaluacion = 90 #asumimos 90 días
periodo_evaluacion = np.linspace(0, dias_evaluacion, dias_evaluacion + 1)

alpha = 0.2
beta = 1.75
gamma = 0.5
rho = 0.82
kappa = .016
parametros = alpha, beta, gamma, rho , kappa

JAL_Population = 8000000 
I_o = 32 / JAL_Population  # Tenemos 32 casos
E_o = (32*4)/JAL_Population # Asumimos 4 expuestos por caso
A_o =  4 / JAL_Population
S_o = (1) - (E_o+I_o+A_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado
M_o = 0 # NO Muertos

Condiciones_Iniciales = S_o,E_o,A_o,I_o,R_o,M_o

In [0]:
def SEAIR_D(IC,pmtrs,t,theta=1):
  """
  Esta parte de código es la que implementa las ecuaciones diferenciales.

  Se resuelven por medio de integración númerica de Euler
  """
  
  So,Eo,Ao,Io,Ro,Do = IC
  S,E,A,I,R,D = [So],[Eo],[Eo],[Io],[Ro],[Do]
  alpha,beta,gamma,rho,kappa = pmtrs
  dt = t[1]-t[0]
  for _ in t[1:]:
    St = S[-1] - (theta*beta*S[-1]*I[-1])*dt
    Et = E[-1] + (theta*beta*S[-1]*I[-1] - alpha*E[-1])*dt
    At = A[-1] + (1-rho)*(alpha*E[-1] - gamma*A[-1])*dt
    It = I[-1] + rho*(alpha*E[-1] - gamma*I[-1])*dt
    Dt = kappa*I[-1]
    Rt = 1 - (S[-1]+E[-1]+A[-1]+I[-1]-D[-1])
    
    S.append(St)
    E.append(Et)
    A.append(At)
    I.append(It)
    R.append(Rt)
    D.append(Dt)
  return S,E,A,I,R,D

In [0]:
S,E,A,I,R,D = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion)

In [0]:
#@title Escenario contabilizando posibles muertes

D_a = np.array(D)*JAL_Population
I_a = np.array(I)*JAL_Population
A_a = np.array(A)*JAL_Population
Clase = np.array(["Muertes"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ ["Asintomaticos o No reportados"]*np.array(E).shape[0])
Dias = np.concatenate([np.arange(0,91,1),np.arange(0,91,1),np.arange(0,91,1)])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([D_a,I_a,A_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase')

fig.update_layout(
    title="Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Mexico",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

En el peor escenario (sin medidas de aislamiento) se esperarían $14,000$ muertes, considerando una tasa de mortalidad fija del $1.6\%$.

Por otro lado, la pregunta de cuantos de los infectados necesitarán atencíon y cuidado hospitalario, como camas en cuidados intensivos, depende también de un porcentaje.

Si tomamos en cuenta un $2\%$ de casos graves, en el peor de los escenarios (sin distanciamiento social) tendriamos la siguiente curva.

In [0]:
plt.figure(figsize= (16,6))
maximo = np.round(np.max(np.array(I)*.02)*JAL_Population,0)
plt.title(f"Máximo de casos graves: {maximo}")
plt.plot(np.array(I)*.02*JAL_Population, label = "Casos graves",c="#FF0000") #Asumimos el 2% de los infectados, de la poblacion de jalisco
plt.ylabel("Casos")
plt.xlabel("Dias")
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

La pregunta es **¿Con cuantas camas de terapia intensiva cuenta el estado?**


Y recalcar de nuevo la importancia de atender las recomendaciones sanitarias, los números de casos graves y muertes, dependen **DIRECTAMENTE** de $\theta$.

Es decir, el distanciamiento social disminuye considerablemente ambos números, porfavor, quedemonos en casa lo más que podamos estos días.

Cuidense ustedes y cuiden a los demás.

<h1><center>Distintos Escenarios</center></h1>




# Condiciones iniciales

In [0]:
dias_evaluacion = 120 #asumimos 120 días
periodo_evaluacion = np.linspace(0, dias_evaluacion, dias_evaluacion + 1)
alpha = 0.2

#Escenarios
def Base():
    Ro = 3.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
def Pesimista():
    Ro = 4.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
def Optimista():
    Ro = 1.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta

def Fuerte():
    Ro = 3.5*(0.5)
    gamma = 0.5
    Beta = Ro*gamma
    return Beta

def Moderada():
    Ro = 3.5 - (3.5*(0.25))
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
switcher = {
   "Base": Base,
   "Pesimista": Pesimista,
   "Optimista": Optimista,
   "Fuerte": Fuerte,
   "Moderada": Moderada
    }
 
def get_Beta(argument):
# Get the function from switcher dictionary
  func = switcher.get(argument, "nothing")
# Execute the function
  return func()

gamma = 0.5
rho = 0.82
kappa = .016
parametros = alpha, beta, gamma, rho , kappa
Population = 8000000 
I_o = 32 / Population  # Tenemos 32 casos
E_o = (32*4)/ Population # Asumimos 4 expuestos por caso
A_o =  4 / Population
D_o = 0 # No Muertos
S_o = (1) - (E_o+I_o+A_o+D_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado


Condiciones_Iniciales = S_o,E_o,A_o,I_o,R_o,D_o


In [0]:
def SEAIR_D(IC, pmtrs, t, intervencion = "Base", inicio_intervecion=1, duracion_intervencion=120):
  """
  Esta parte de código es la que implementa las ecuaciones diferenciales.

  Se resuelven por medio de integración númerica de Euler
  """
  
  So,Eo,Ao,Io,Ro,Do = IC
  S,E,A,I,R,D = [So],[Eo],[Eo],[Io],[Ro],[Do]
  alpha,beta,gamma,rho,kappa = pmtrs
  dt = t[1]-t[0]
  for i in t[1:]:
    if (i >= inicio_intervecion and i <=duracion_intervencion):
      beta = get_Beta(intervencion)
    else:
      beta = get_Beta("Base")


    St = S[-1] - (beta*S[-1]*I[-1])*dt
    Et = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
    At = A[-1] + (1-rho)*(alpha*E[-1] - gamma*A[-1])*dt
    It = I[-1] + rho*(alpha*E[-1] - gamma*I[-1])*dt
    Dt = kappa*I[-1]
    Rt = 1 - (S[-1]+E[-1]+A[-1]+I[-1]-D[-1])
    
    S.append(St)
    E.append(Et)
    A.append(At)
    I.append(It)
    R.append(Rt)
    D.append(Dt)
  return S,E,A,I,R,D, intervencion

In [0]:
S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Pesimista",1,120)
                                                                                        #Escenario,#Dia que inicia la intervencion,#Cuantos dias dura

In [40]:
#@title Escenario Base: Ro = 3.5, no intervencion { display-mode: "form" }

S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Base",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

In [37]:
#@title Escenario Optimista: Ro = 1.5, no intervencion { display-mode: "form" }

S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Optimista",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

In [38]:
#@title Escenario Pesimista: Ro = 4.5, no intervencion { display-mode: "form" }

S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Pesimista",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

El escenario de intervención "Fuerte" incluye:

* Cierre de negocios
* Cierre de transporte
* Cierre de escuelas
* Pruebas masivas a conductores
* Distanciamiento social
* Casi todo cerrado

(Wu et al., 2020)

In [47]:
#@title Escenario Base: Ro = 1.75, Intervencion "FUERTE" Constante (120 Dias) { display-mode: "form" }


S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Fuerte",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

Escenario moderado incluye:

* Distanciamiento social
* Cierre de escuelas

(Wu et al., 2020)

In [46]:
#@title Escenario Moderada: Ro = 2.62, Intervencion Constante (120 Dias) { display-mode: "form" }

S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Moderada",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

# Casos Graves 

Tomando en cuenta que de los infectados se espera el $2\%$ grave

In [53]:
#@title Distintos escenarios de pacientes graves { display-mode: "form" }


_,_,_,I_base,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Base",1,120)
_,_,_,I_opt,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Optimista",1,120)
_,_,_,I_pes,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Pesimista",1,120)
_,_,_,I_Fuerte,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Fuerte",1,120)
_,_,_,I_Moderada,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Moderada",1,120)


I_base = np.array(I_base)*Population * .02
I_opt = np.array(I_opt)*Population * .02
I_pes = np.array(I_pes)*Population * .02
I_Fuerte = np.array(I_Fuerte)*Population * .02
I_Moderada = np.array(I_Moderada)*Population * .02
Clase = np.array(["Graves - Pesimista"]*np.array(I_base).shape[0] + ["Graves - Base"]*np.array(I_base).shape[0]+ 
                  ["Graves - Intervencion Moderada"]*np.array(I_base).shape[0] + 
                 ["Graves - Interveción Fuerte"]*np.array(I_base).shape[0]+ ["Graves - Optimista"]*np.array(I_base).shape[0])

Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([I_pes,I_base,I_Moderada,I_Fuerte,I_opt]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase', color_discrete_sequence=px.colors.sequential.RdBu, template = "ggplot2")

fig.update_layout(
    title="Predicción del casos graves de COVID-19 en Jalisco, en distintos Escenarios",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

In [52]:
print(px.colors.sequential.RdBu)

['rgb(103,0,31)', 'rgb(178,24,43)', 'rgb(214,96,77)', 'rgb(244,165,130)', 'rgb(253,219,199)', 'rgb(247,247,247)', 'rgb(209,229,240)', 'rgb(146,197,222)', 'rgb(67,147,195)', 'rgb(33,102,172)', 'rgb(5,48,97)']
